# CUB

In [2]:
import os, glob, shutil
import numpy as np

classes = np.arange(1,201)

train = np.random.choice(classes, 128, False)

val_test = set(classes).difference(set(train))
val_test = np.array(list(val_test))

valid = np.random.choice(val_test, 32, False)

test = set(val_test).difference(set(valid))
test = np.array(list(test))


train_class = []
for i in train:
    if i < 10:
        num = '00'+str(i)
    elif i >=10 and i < 100:
        num = '0'+str(i)
    else:
        num = str(i)
    train_class.append(num)

valid_class = []
for i in valid:
    if i < 10:
        num = '00'+str(i)
    elif i >=10 and i < 100:
        num = '0'+str(i)
    else:
        num = str(i)
    valid_class.append(num)

test_class = []
for i in test:
    if i < 10:
        num = '00'+str(i)
    elif i >=10 and i < 100:
        num = '0'+str(i)
    else:
        num = str(i)
    test_class.append(num)


path = '/home/hjb3880/WORKPLACE/datasets/CUB_200_2011/images'
new_path = '/home/hjb3880/WORKPLACE/datasets/CUB_200_2011'
all_dir_list = glob.glob(f'{path}/*')

for dir in all_dir_list:
    new_folder_name = dir.split('/')[-1].replace('.', '_')
    if dir.split('/')[-1][:3] in train_class:
        new_dir = os.path.join(new_path, 'train', new_folder_name)
    elif dir.split('/')[-1][:3] in valid_class:
        new_dir = os.path.join(new_path, 'validation', new_folder_name)
    elif dir.split('/')[-1][:3] in test_class:
        new_dir = os.path.join(new_path, 'test', new_folder_name)
    
    shutil.copytree(dir, new_dir)

---

# Aircraft

In [131]:
import os, shutil, random

def seed_fixer(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_fixer(2023)


# train-valid-test split about classes (64:16:20)
variants_path = './Aircraft/data/variants.txt'
source_dir = '/home/hjb3880/WORKPLACE/datasets/Aircraft/data/images'
target_dir = f'/home/hjb3880/WORKPLACE/datasets/Aircraft'

labels = set()
with open(variants_path, 'r') as f:
    for line in f:
        label = '-'.join(line.strip().split())
        label = label.replace('/','-')
        labels.add(label)

train_label = set(random.sample(labels, 64))
vat_test_labels = labels - train_label

val_label = set(random.sample(vat_test_labels, 16))
test_label = vat_test_labels - val_label


# combin images_variant_train, _val, _test
all_text = []
mode = ['train', 'val', 'test']
for m in mode:
    text_path = f'./Aircraft/data/images_variant_{m}.txt'
    with open(text_path, 'r') as f:
        for line in f:
            name_label = line.strip().split()
            name = name_label[0]+'.jpg'
            if len(name_label) == 2 :
                label = name_label[1]
            elif len(name_label) >= 3 :
                label = '-'.join(name_label[1:])
            label = label.replace('/','-')
            all_text.append([name, label])
    

    split_path = os.path.join(target_dir, m)
    if not os.path.exists(split_path):
        os.mkdir(split_path)


# move images
for n, l in all_text:
    source_path = os.path.join(source_dir, n)
    
    if l in train_label:
        label_path = os.path.join(target_dir, 'train', l)
    elif l in val_label:
        label_path = os.path.join(target_dir, 'val', l)
    elif l in test_label:
        label_path = os.path.join(target_dir, 'test', l)

    if not os.path.exists(label_path):
        os.mkdir(label_path)
    target_path = os.path.join(label_path, n)

    try:
        shutil.move(source_path, target_path)
    except FileNotFoundError:
        print(f"'{n}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일 이동 중 오류가 발생했습니다: {e}")
        